# FinSim Price Forecasting with LSTM, Transformer & GRU

This notebook demonstrates advanced price prediction techniques using deep learning models in the FinSim platform.

## Overview
- LSTM (Long Short-Term Memory) networks for sequential pattern recognition
- Transformer models with attention mechanisms for complex temporal relationships
- GRU (Gated Recurrent Unit) networks for efficient sequence processing

## References
- Hochreiter, S., & Schmidhuber, J. "Long Short-Term Memory." Neural Computation, 1997.
- Vaswani, A. et al. "Attention Is All You Need." NeurIPS, 2017.

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# FinSim API configuration
FINSIM_API_BASE = "http://localhost:8000/api/v1"
SYMBOLS = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'NVDA']

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Data Collection and Preprocessing

In [ ]:
def fetch_market_data(symbol, period='2y'):
    """Fetch historical market data"""
    try:
        # Try FinSim API first
        response = requests.post(f"{FINSIM_API_BASE}/historical", 
                               json={"symbols": [symbol], "interval": "1d"})
        if response.status_code == 200:
            data = response.json()[symbol]['data']
            return pd.DataFrame(data)
    except:
        pass
    
    # Fallback to Yahoo Finance
    ticker = yf.Ticker(symbol)
    data = ticker.history(period=period)
    return data.reset_index()

def prepare_features(df):
    """Create technical indicators and features"""
    df = df.copy()
    
    # Price features
    df['returns'] = df['Close'].pct_change()
    df['log_returns'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # Technical indicators
    df['sma_5'] = df['Close'].rolling(5).mean()
    df['sma_20'] = df['Close'].rolling(20).mean()
    df['sma_50'] = df['Close'].rolling(50).mean()
    
    # Volatility
    df['volatility'] = df['returns'].rolling(20).std()
    
    # Volume indicators
    df['volume_sma'] = df['Volume'].rolling(20).mean()
    df['volume_ratio'] = df['Volume'] / df['volume_sma']
    
    # RSI
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    df['bb_upper'] = df['sma_20'] + (df['Close'].rolling(20).std() * 2)
    df['bb_lower'] = df['sma_20'] - (df['Close'].rolling(20).std() * 2)
    df['bb_position'] = (df['Close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])
    
    return df.dropna()

# Collect data for all symbols
data_dict = {}
for symbol in SYMBOLS:
    print(f"Fetching data for {symbol}...")
    df = fetch_market_data(symbol)
    df = prepare_features(df)
    data_dict[symbol] = df
    print(f"  {len(df)} records collected")

print("\nData collection completed!")

## 2. LSTM Model Implementation

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=50, num_layers=2, dropout=0.2, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])  # Take last time step
        out = self.fc(out)
        return out

def create_sequences(data, seq_length=20):
    """Create sequences for training"""
    sequences = []
    targets = []
    
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+seq_length]
        sequences.append(seq)
        targets.append(target)
    
    return np.array(sequences), np.array(targets)

def train_lstm_model(symbol, data_dict, seq_length=20, epochs=100):
    """Train LSTM model for a specific symbol"""
    df = data_dict[symbol]
    
    # Select features
    feature_cols = ['Close', 'Volume', 'returns', 'sma_5', 'sma_20', 'volatility', 'rsi', 'bb_position']
    features = df[feature_cols].values
    
    # Normalize features
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Create sequences
    X, y = create_sequences(features_scaled, seq_length)
    
    # Split train/test
    split_idx = int(0.8 * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    # Convert to tensors
    X_train = torch.FloatTensor(X_train)
    y_train = torch.FloatTensor(y_train[:, 0])  # Predict Close price
    X_test = torch.FloatTensor(X_test)
    y_test = torch.FloatTensor(y_test[:, 0])
    
    # Initialize model
    model = LSTMModel(input_size=len(feature_cols))
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    train_losses = []
    model.train()
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        if (epoch + 1) % 20 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.6f}")
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        train_pred = model(X_train).squeeze()
        test_pred = model(X_test).squeeze()
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train.numpy(), train_pred.numpy())
    test_mse = mean_squared_error(y_test.numpy(), test_pred.numpy())
    train_mae = mean_absolute_error(y_train.numpy(), train_pred.numpy())
    test_mae = mean_absolute_error(y_test.numpy(), test_pred.numpy())
    
    print(f"\nLSTM Results for {symbol}:")
    print(f"Train MSE: {train_mse:.6f}, Test MSE: {test_mse:.6f}")
    print(f"Train MAE: {train_mae:.6f}, Test MAE: {test_mae:.6f}")
    
    return {
        'model': model,
        'scaler': scaler,
        'train_losses': train_losses,
        'predictions': {
            'train': train_pred.numpy(),
            'test': test_pred.numpy(),
            'y_train': y_train.numpy(),
            'y_test': y_test.numpy()
        },
        'metrics': {
            'train_mse': train_mse,
            'test_mse': test_mse,
            'train_mae': train_mae,
            'test_mae': test_mae
        }
    }

# Train LSTM for AAPL
print("Training LSTM model for AAPL...")
lstm_results = train_lstm_model('AAPL', data_dict)

## 3. Transformer Model Implementation

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.d_k)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def forward(self, x):
        batch_size, seq_length, d_model = x.size()
        
        Q = self.W_q(x).view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
        attn_output = self.scaled_dot_product_attention(Q, K, V)
        attn_output = attn_output.transpose(1, 2).contiguous().view(
            batch_size, seq_length, d_model)
        
        output = self.W_o(attn_output)
        return output

class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        attn_output = self.attention(x)
        x = self.norm1(x + self.dropout(attn_output))
        
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        
        return x

class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model=128, num_heads=8, num_layers=3, d_ff=512, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.d_model = d_model
        self.input_projection = nn.Linear(input_size, d_model)
        self.positional_encoding = self.create_positional_encoding(1000, d_model)
        
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        self.ln_f = nn.LayerNorm(d_model)
        self.fc = nn.Linear(d_model, 1)
        self.dropout = nn.Dropout(dropout)
        
    def create_positional_encoding(self, max_seq_length, d_model):
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                           (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)
        
    def forward(self, x):
        seq_length = x.size(1)
        x = self.input_projection(x) * np.sqrt(self.d_model)
        x = x + self.positional_encoding[:, :seq_length, :].to(x.device)
        x = self.dropout(x)
        
        for transformer in self.transformer_blocks:
            x = transformer(x)
            
        x = self.ln_f(x)
        x = self.fc(x[:, -1, :])  # Take last time step
        return x

def train_transformer_model(symbol, data_dict, seq_length=20, epochs=100):
    """Train Transformer model for a specific symbol"""
    df = data_dict[symbol]
    
    # Select features
    feature_cols = ['Close', 'Volume', 'returns', 'sma_5', 'sma_20', 'volatility', 'rsi', 'bb_position']
    features = df[feature_cols].values
    
    # Normalize features
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Create sequences
    X, y = create_sequences(features_scaled, seq_length)
    
    # Split train/test
    split_idx = int(0.8 * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    # Convert to tensors
    X_train = torch.FloatTensor(X_train)
    y_train = torch.FloatTensor(y_train[:, 0])  # Predict Close price
    X_test = torch.FloatTensor(X_test)
    y_test = torch.FloatTensor(y_test[:, 0])
    
    # Initialize model
    model = TransformerModel(input_size=len(feature_cols))
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    train_losses = []
    model.train()
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        if (epoch + 1) % 20 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.6f}")
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        train_pred = model(X_train).squeeze()
        test_pred = model(X_test).squeeze()
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train.numpy(), train_pred.numpy())
    test_mse = mean_squared_error(y_test.numpy(), test_pred.numpy())
    train_mae = mean_absolute_error(y_train.numpy(), train_pred.numpy())
    test_mae = mean_absolute_error(y_test.numpy(), test_pred.numpy())
    
    print(f"\nTransformer Results for {symbol}:")
    print(f"Train MSE: {train_mse:.6f}, Test MSE: {test_mse:.6f}")
    print(f"Train MAE: {train_mae:.6f}, Test MAE: {test_mae:.6f}")
    
    return {
        'model': model,
        'scaler': scaler,
        'train_losses': train_losses,
        'predictions': {
            'train': train_pred.numpy(),
            'test': test_pred.numpy(),
            'y_train': y_train.numpy(),
            'y_test': y_test.numpy()
        },
        'metrics': {
            'train_mse': train_mse,
            'test_mse': test_mse,
            'train_mae': train_mae,
            'test_mae': test_mae
        }
    }

# Train Transformer for AAPL
print("Training Transformer model for AAPL...")
transformer_results = train_transformer_model('AAPL', data_dict)

## 4. GRU Model Implementation

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size=50, num_layers=2, dropout=0.2, output_size=1):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers, 
                         dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])  # Take last time step
        out = self.fc(out)
        return out

def train_gru_model(symbol, data_dict, seq_length=20, epochs=100):
    """Train GRU model for a specific symbol"""
    df = data_dict[symbol]
    
    # Select features
    feature_cols = ['Close', 'Volume', 'returns', 'sma_5', 'sma_20', 'volatility', 'rsi', 'bb_position']
    features = df[feature_cols].values
    
    # Normalize features
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Create sequences
    X, y = create_sequences(features_scaled, seq_length)
    
    # Split train/test
    split_idx = int(0.8 * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    # Convert to tensors
    X_train = torch.FloatTensor(X_train)
    y_train = torch.FloatTensor(y_train[:, 0])  # Predict Close price
    X_test = torch.FloatTensor(X_test)
    y_test = torch.FloatTensor(y_test[:, 0])
    
    # Initialize model
    model = GRUModel(input_size=len(feature_cols))
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    train_losses = []
    model.train()
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        if (epoch + 1) % 20 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.6f}")
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        train_pred = model(X_train).squeeze()
        test_pred = model(X_test).squeeze()
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train.numpy(), train_pred.numpy())
    test_mse = mean_squared_error(y_test.numpy(), test_pred.numpy())
    train_mae = mean_absolute_error(y_train.numpy(), train_pred.numpy())
    test_mae = mean_absolute_error(y_test.numpy(), test_pred.numpy())
    
    print(f"\nGRU Results for {symbol}:")
    print(f"Train MSE: {train_mse:.6f}, Test MSE: {test_mse:.6f}")
    print(f"Train MAE: {train_mae:.6f}, Test MAE: {test_mae:.6f}")
    
    return {
        'model': model,
        'scaler': scaler,
        'train_losses': train_losses,
        'predictions': {
            'train': train_pred.numpy(),
            'test': test_pred.numpy(),
            'y_train': y_train.numpy(),
            'y_test': y_test.numpy()
        },
        'metrics': {
            'train_mse': train_mse,
            'test_mse': test_mse,
            'train_mae': train_mae,
            'test_mae': test_mae
        }
    }

# Train GRU for AAPL
print("Training GRU model for AAPL...")
gru_results = train_gru_model('AAPL', data_dict)

## 5. Model Comparison and Visualization

In [ ]:
# Compare model performance
models = {
    'LSTM': lstm_results,
    'Transformer': transformer_results,
    'GRU': gru_results
}

# Create comparison dataframe
comparison_data = []
for model_name, results in models.items():
    comparison_data.append({
        'Model': model_name,
        'Train MSE': results['metrics']['train_mse'],
        'Test MSE': results['metrics']['test_mse'],
        'Train MAE': results['metrics']['train_mae'],
        'Test MAE': results['metrics']['test_mae']
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nModel Comparison:")
print(comparison_df.round(6))

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Price Prediction Models Comparison - AAPL', fontsize=16, fontweight='bold')

# Training losses
axes[0, 0].set_title('Training Losses')
for model_name, results in models.items():
    axes[0, 0].plot(results['train_losses'], label=model_name, alpha=0.8)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('MSE Loss')
axes[0, 0].legend()
axes[0, 0].set_yscale('log')
axes[0, 0].grid(True, alpha=0.3)

# Test predictions comparison
axes[0, 1].set_title('Test Set Predictions')
test_indices = range(len(lstm_results['predictions']['y_test']))
axes[0, 1].plot(test_indices, lstm_results['predictions']['y_test'], 
                label='Actual', color='black', linewidth=2)
for model_name, results in models.items():
    axes[0, 1].plot(test_indices, results['predictions']['test'], 
                    label=f'{model_name} Pred', alpha=0.8)
axes[0, 1].set_xlabel('Time Steps')
axes[0, 1].set_ylabel('Normalized Price')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Model performance metrics
metrics = ['Train MSE', 'Test MSE', 'Train MAE', 'Test MAE']
x = np.arange(len(metrics))
width = 0.25

for i, (model_name, results) in enumerate(models.items()):
    values = [results['metrics']['train_mse'], results['metrics']['test_mse'],
             results['metrics']['train_mae'], results['metrics']['test_mae']]
    axes[1, 0].bar(x + i*width, values, width, label=model_name, alpha=0.8)

axes[1, 0].set_title('Performance Metrics Comparison')
axes[1, 0].set_xlabel('Metrics')
axes[1, 0].set_ylabel('Error Value')
axes[1, 0].set_xticks(x + width)
axes[1, 0].set_xticklabels(metrics, rotation=45)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Prediction accuracy scatter plot
axes[1, 1].set_title('Prediction vs Actual (Test Set)')
for model_name, results in models.items():
    axes[1, 1].scatter(results['predictions']['y_test'], 
                      results['predictions']['test'], 
                      alpha=0.6, label=model_name, s=20)

# Perfect prediction line
min_val = min([min(results['predictions']['y_test']) for results in models.values()])
max_val = max([max(results['predictions']['y_test']) for results in models.values()])
axes[1, 1].plot([min_val, max_val], [min_val, max_val], 
                'r--', alpha=0.8, label='Perfect Prediction')

axes[1, 1].set_xlabel('Actual Values')
axes[1, 1].set_ylabel('Predicted Values')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print best model
best_model = comparison_df.loc[comparison_df['Test MSE'].idxmin(), 'Model']
print(f"\n🏆 Best performing model: {best_model}")
print(f"Test MSE: {comparison_df.loc[comparison_df['Test MSE'].idxmin(), 'Test MSE']:.6f}")

## 6. Live Prediction with FinSim Integration

In [ ]:
def deploy_model_to_finsim(model_results, symbol, model_type):
    """Deploy trained model as a FinSim agent"""
    
    agent_config = {
        "agent_id": f"{model_type}_predictor_{symbol}",
        "agent_type": model_type.lower(),
        "symbols": [symbol],
        "parameters": {
            "model_path": f"models/{model_type}_{symbol}.pth",
            "sequence_length": 20,
            "confidence_threshold": 0.6
        },
        "enabled": True
    }
    
    try:
        response = requests.post(f"{FINSIM_API_BASE}/agents", json=agent_config)
        if response.status_code == 200:
            print(f"✅ {model_type} agent deployed successfully for {symbol}")
            return response.json()
        else:
            print(f"❌ Failed to deploy {model_type} agent: {response.text}")
    except Exception as e:
        print(f"❌ Error deploying agent: {e}")
    
    return None

def make_live_prediction(model_results, symbol, model_type):
    """Make live predictions using trained model"""
    model = model_results['model']
    scaler = model_results['scaler']
    
    # Get latest market data
    try:
        response = requests.get(f"{FINSIM_API_BASE}/quotes/{symbol}")
        if response.status_code == 200:
            quote = response.json()
            
            # Prepare features (simplified for demo)
            current_features = np.array([
                quote['price'],
                quote['volume'],
                0.0,  # returns (would need historical data)
                quote['price'],  # sma_5 (simplified)
                quote['price'],  # sma_20 (simplified)
                0.02,  # volatility (estimated)
                50.0,  # rsi (neutral)
                0.5   # bb_position (middle)
            ]).reshape(1, -1)
            
            # Normalize features
            current_features_scaled = scaler.transform(current_features)
            
            # Create sequence (repeat current features for sequence length)
            sequence = np.repeat(current_features_scaled, 20, axis=0).reshape(1, 20, -1)
            
            # Make prediction
            model.eval()
            with torch.no_grad():
                prediction = model(torch.FloatTensor(sequence))
                predicted_price = prediction.item()
            
            # Calculate prediction confidence and direction
            current_price_normalized = current_features_scaled[0, 0]
            price_change = predicted_price - current_price_normalized
            direction = "UP" if price_change > 0 else "DOWN"
            confidence = min(abs(price_change) * 100, 100)  # Simplified confidence
            
            return {
                'symbol': symbol,
                'model': model_type,
                'current_price': quote['price'],
                'predicted_direction': direction,
                'confidence': confidence,
                'prediction_raw': predicted_price,
                'timestamp': datetime.now().isoformat()
            }
            
    except Exception as e:
        print(f"Error making live prediction: {e}")
    
    return None

# Deploy best model to FinSim
print("\nDeploying models to FinSim...")
for model_name, results in models.items():
    deploy_model_to_finsim(results, 'AAPL', model_name)

# Make live predictions
print("\nMaking live predictions...")
live_predictions = []
for model_name, results in models.items():
    prediction = make_live_prediction(results, 'AAPL', model_name)
    if prediction:
        live_predictions.append(prediction)

# Display predictions
if live_predictions:
    predictions_df = pd.DataFrame(live_predictions)
    print("\nLive Price Predictions:")
    print(predictions_df[['model', 'current_price', 'predicted_direction', 'confidence']].round(2))
else:
    print("No live predictions available (API not accessible)")

print("\n" + "="*50)
print("PREDICTION NOTEBOOK COMPLETED SUCCESSFULLY!")
print("Models trained:", list(models.keys()))
print(f"Best model: {best_model}")
print("="*50)